In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from scipy.signal import butter, lfilter, filtfilt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import keras
from keras.layers import Dropout
from keras.layers import Activation
from shutil import copyfile
import gc
import os
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as signal
kf = KFold(n_splits=5, shuffle= True, random_state=28)
from scipy import stats
from sklearn.model_selection import train_test_split
import librosa
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import keras
from keras.layers import Dropout
from keras.layers import Activation
from shutil import copyfile
import gc
import os
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as signal
kf = KFold(n_splits=5, shuffle= True, random_state=28)
from scipy import stats
import python_speech_features
from essentia.standard import *
from sklearn.model_selection import train_test_split

Tải dữ liệu từ drive vào Goolge Colab

In [0]:
!cp /content/drive/My\ Drive/DuLieu/filter_feature/m.npy .
!cp /content/drive/My\ Drive/DuLieu/filter_feature/m_l.npy .
!cp /content/drive/My\ Drive/DuLieu/filter_feature/m_a.npy .
!cp /content/drive/My\ Drive/DuLieu/filter_feature/m_a_l.npy .

In [0]:
class LogConfusionMatrix(keras.callbacks.Callback):
  def __init__(self, names):
    self.names = names
  def on_train_begin(self, logs={}):
    self.bestacc = 0.0
    self.bestcf = None
  def on_epoch_end(self, epoch, logs = {}):
    if logs["val_acc"] > self.bestacc:
      self.bestacc = logs["val_acc"] 
      y_pred = self.model.predict(self.validation_data[0])
      y_pred = [1 * (x[0]>=0.5) for x in y_pred]
      self.bestcf = confusion_matrix(self.validation_data[1], y_pred)
  def on_train_end(self, logs = {}):
    f = open(self.names, "a")
    f.write("Best val_acc: " + str(self.bestacc))
    f.write("\n")
    f.write("Best cf: \n" + str(self.bestcf))
    f.write("\n")
    f.close()

In [0]:
def create_model():
  model = Sequential()
  model.add(LSTM(64, return_sequences=True))
  model.add(LSTM(64))
  model.add(Dense(1, activation = "sigmoid"))
  model.compile(loss='binary_crossentropy', optimizer= keras.optimizers.Adam(lr = 0.0005), metrics=['accuracy'])
  return model

In [0]:
def train_case_1(X_aug, y_aug, filename, epochs = 40):
  i = 1
  for train_index, test_index in kf.split(X_aug):
    print("Cross Valid: " + str(i))
    f = open(filename, "a")
    f.write("Cross Valid: " + str(i))
    f.write("\n")
    f.close()
    logcf = LogConfusionMatrix(filename)
    es = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.001, patience=15, verbose=1, mode='auto')
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = "loss",patience = 5, factor = 0.9, min_lr = 0.0001, verbose = 1)
    model = create_model()
    mean = np.mean(X_aug[train_index], axis = 0)
    std = np.std(X_aug[train_index], axis = 0)
    X_train = (X_aug[train_index] - mean) / std
    y_train = y_aug[train_index]
    X_test = (X_aug[test_index] - mean) / std
    y_test = y_aug[test_index]
    model.fit(X_train, y_train, epochs = epochs, batch_size = 64, verbose = 1, validation_data = (X_test, y_test), callbacks = [es, logcf, reduce_lr])
    #print(confusion_matrix(y_test, y_pred))
    gc.collect()
    i += 1

In [0]:
X = np.load("m.npy")
y = np.load("m_l.npy")

In [10]:
train_case_1(X, y, "mfcc_case1.txt", 50)

Cross Valid: 1





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 4393 samples, validate on 1099 samples
Epoch 1/50





4393/4393 [==============================] - 16s 4ms/step - loss: 0.5088 - acc: 0.7683 - val_loss: 0.4078 - val_acc: 0.8344
Epoch 2/50
4393/4393 [==============================] - 13s 3ms/step - loss: 0.3447 - acc: 0.8712 - val_loss: 0.3189 - val_acc: 0.8881
Epoch 3/50
4393/4393 [==============================] - 14s 3ms/step - loss: 0.2794 - acc: 0.8998 - val_loss: 0.2901 - val_acc: 0.8908
Epoch 4/50
4393/4393 [==============================] - 14s 3ms/step - loss: 0.2393 - acc: 0.9158 - val_loss: 0.2614 - val_acc: 0.9090
Epoch 5/50
4393/4393 [==============================] - 13s 3ms/step - loss: 0.2025 - acc: 0.9363 - val_loss: 0.2904 - val_acc: 0.8926
Epoch 6/50
4393/4393 [==============================] - 14s 3ms/step - loss: 0.1839 - acc: 0.9369 - val_loss: 0.2419 - val_acc: 0.9190
Epoch 7/50
4393/4393

In [0]:
def train_case_3(X_train, y_train, X_test, y_test,filename, epochs = 50):
  model = create_model()
  logcf = LogConfusionMatrix(filename)
  es = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.008, patience=9, verbose=1, mode='auto')
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = "loss",patience = 3, factor = 0.8, min_lr = 0.0001, verbose = 1)
  model.fit(X_train, y_train, epochs = epochs, batch_size = 64, validation_data = (X_test, y_test), callbacks = [es, logcf, reduce_lr])

In [19]:
X_train = np.load("m_a.npy")
y_train = np.load("m_a_l.npy")
X_test = np.load("m.npy")
y_test = np.load("m_l.npy")
mean = np.mean(X_train, axis = 0)
std = np.std(X_train, axis = 0)
X_train = stats.zscore(X_train)
X_test = (X_test - mean)/std
train_case_3(X_train, y_train, X_test, y_test, filename = "mfcc_case_3.txt", epochs = 50)

Train on 38710 samples, validate on 5492 samples
Epoch 1/50
38710/38710 [==============================] - 122s 3ms/step - loss: 0.3055 - acc: 0.8739 - val_loss: 0.1390 - val_acc: 0.9541
Epoch 2/50
38710/38710 [==============================] - 118s 3ms/step - loss: 0.1418 - acc: 0.9520 - val_loss: 0.1046 - val_acc: 0.9652
Epoch 3/50
38710/38710 [==============================] - 117s 3ms/step - loss: 0.0843 - acc: 0.9725 - val_loss: 0.0653 - val_acc: 0.9778
Epoch 4/50
38710/38710 [==============================] - 118s 3ms/step - loss: 0.0581 - acc: 0.9814 - val_loss: 0.0512 - val_acc: 0.9834
Epoch 5/50
38710/38710 [==============================] - 117s 3ms/step - loss: 0.0392 - acc: 0.9874 - val_loss: 0.0432 - val_acc: 0.9869
Epoch 6/50
38710/38710 [==============================] - 122s 3ms/step - loss: 0.0305 - acc: 0.9898 - val_loss: 0.0329 - val_acc: 0.9885
Epoch 7/50
38710/38710 [==============================] - 120s 3ms/step - loss: 0.0236 - acc: 0.9921 - val_loss: 0.0334 - v

In [0]:
def train_case_4(X_train, y_train, X_test, y_test,filename, epochs = 50):
  model = create_model()
  logcf = LogConfusionMatrix(filename)
  es = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.001, patience=15, verbose=1, mode='auto')
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = "loss",patience = 5, factor = 0.9, min_lr = 0.0001, verbose = 1)
  model.fit(X_train, y_train, epochs = epochs, batch_size = 64, validation_data = (X_test, y_test), callbacks = [es, logcf, reduce_lr])

In [17]:
X_train = np.load("m.npy")
y_train = np.load("m_l.npy")
X_test = np.load("m_a.npy")
y_test = np.load("m_a_l.npy")
mean = np.mean(X_train, axis = 0)
std = np.std(X_train, axis = 0)
X_train = stats.zscore(X_train)
X_test = (X_test - mean)/std
train_case_4(X_train, y_train, X_test, y_test, filename = "mfcc_case_4.txt", epochs = 50)

Train on 5492 samples, validate on 38710 samples
Epoch 1/50
5492/5492 [==============================] - 44s 8ms/step - loss: 0.4744 - acc: 0.7999 - val_loss: 0.5770 - val_acc: 0.7584
Epoch 2/50
5492/5492 [==============================] - 38s 7ms/step - loss: 0.3146 - acc: 0.8840 - val_loss: 0.3952 - val_acc: 0.8351
Epoch 3/50
5492/5492 [==============================] - 38s 7ms/step - loss: 0.2563 - acc: 0.9080 - val_loss: 0.3718 - val_acc: 0.8626
Epoch 4/50
5492/5492 [==============================] - 38s 7ms/step - loss: 0.2069 - acc: 0.9299 - val_loss: 0.3218 - val_acc: 0.8819
Epoch 5/50
5492/5492 [==============================] - 38s 7ms/step - loss: 0.1712 - acc: 0.9430 - val_loss: 0.2880 - val_acc: 0.8955
Epoch 6/50
5492/5492 [==============================] - 38s 7ms/step - loss: 0.1537 - acc: 0.9492 - val_loss: 0.2516 - val_acc: 0.9099
Epoch 7/50
5492/5492 [==============================] - 38s 7ms/step - loss: 0.1235 - acc: 0.9610 - val_loss: 0.2219 - val_acc: 0.9237
Epoch 